# EGSIS = Exploratory Graph-based Semi Supervised Image Segmentation

Unindo todas as partes: 

- criação de superpixels
- criação de rede complexa
- aplicação da dinâmica coletiva (LCU)
- classificando as regiões não conhecidas
- obtendo segmentos da imagem

Essa variação contém um anotador que desenvolvi para não ser necessário pré-segmentar a imagem com superpixels pra realizar a anotação. Por esse motivo, as gerações prévias dos superpixels e redes complexas serão omitidas.

In [1]:
import os
os.environ["LOGURU_LEVEL"] = "DEBUG"

In [2]:
# ref: https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_segmentations.html

import matplotlib.pyplot as plt
import numpy as np

from skimage.data import astronaut, cat, microaneurysms, rocket, horse, retina, coins
from skimage.util import img_as_ubyte

from egsis import model
from egsis import image
from egsis.annotators import PointAnnotator

# Label annotation

In [3]:
img = img_as_ubyte(retina())

annotator = PointAnnotator(img, classes=["bg", "retina"], resize_f=0.4)
annotator.display()

# Model EGSIS execution

In [ ]:
X = img
y = annotator.label_matrix

m = model.EGSIS(
    superpixel_segments=50,
    superpixel_sigma=0.3,
    superpixel_compactness=40,
    feature_extraction="gabor",
    lcu_max_iter=100,
    feature_similarity="cosine",
)

# Fit predict receiving a image mask
mask = m.fit_predict_segmentation_mask(X, y)
plt.imshow(image.segmentation_mask_blend(img, mask, annotator.labels_by_color))
plt.axis('off')

2023-11-11 21:19:19.266 | INFO     | egsis.model:fit_predict:143 - Run!
